In [ ]:
!pip install transformers

In [4]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('miemBertProject/miem-scibert-linguistic')
model = BertForMaskedLM.from_pretrained('miemBertProject/miem-scibert-linguistic')

input_ids = tokenizer("Сравнительное влияние климата и обновления культивации на урожайность риса в Китае", return_tensors="pt")["input_ids"]
idx_to_predict=11
input_ids[0,idx_to_predict]=tokenizer.mask_token_id
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
    loss, prediction_scores = outputs[:2]
predicted_index = torch.argmax(prediction_scores[0][idx_to_predict]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/520 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/866 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [5]:
predicted_token,tokenizer.convert_ids_to_tokens([tokenizer("Сравнительное влияние климата и обновления культивации на урожайность риса в Китае", return_tensors="pt")["input_ids"][0,idx_to_predict]])

(['риса'], ['риса'])

In [6]:
from transformers import BertModel
model=BertModel.from_pretrained('miemBertProject/miem-scibert-linguistic')
model.eval()
input_ids = tokenizer("Сравнительное влияние климата и обновления культивации на урожайность риса в Китае", return_tensors="pt")["input_ids"]
outputs=model(input_ids)

Some weights of the model checkpoint at miemBertProject/miem-scibert-linguistic were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at miemBertProject/miem-scibert-linguistic and are newly initialized: ['bert.pooler.dens

In [7]:
outputs[0].shape,outputs[1].shape

(torch.Size([1, 15, 768]), torch.Size([1, 768]))

In [ ]:
outputs[0]~[Количество текстов в баче; Количество токенов в тексте; Размер скрытого пространства=768] Эмбеддинги токенов
outputs[1]~[Количество текстов в баче; Размер скрытого пространства=768] Эмбеддинг текста